In [1]:
import dbt
from dbt.config import Profile
from dbt.task.base import read_profiles
from dbt.adapters.snowflake.impl import SnowflakeAdapter

In [2]:
def get_adapter(profile_name, target_name):
    raw_profiles = read_profiles()
    creds = Profile._credentials_from_profile(raw_profiles[profile_name]["outputs"][target_name], profile_name, target_name)
    profile = Profile.from_credentials(creds, 1, profile_name, target_name)
    adapter = SnowflakeAdapter(profile)
    adapter.acquire_connection()
    return adapter

In [5]:
adapter = get_adapter(profile_name="default", target_name="dev")

In [11]:
query = '''
SELECT l_returnflag,
       l_linestatus,
       sum(l_quantity)      as sum_qty,
       sum(l_extendedprice) as sum_base_price,
       sum(l_extendedprice * (1 - l_discount))
                            as sum_disc_price,
       sum(l_extendedprice * (1 - l_discount) *
           (1 + l_tax))     as sum_charge,
       avg(l_quantity)      as avg_qty,
       avg(l_extendedprice) as avg_price,
       avg(l_discount)      as avg_disc,
       count(*)             as count_order
FROM sampledb.tpch_sf100.lineitem
WHERE l_shipdate <= dateadd(day, -90, to_date('1998-12-01'))
GROUP BY l_returnflag,
         l_linestatus
ORDER BY l_returnflag,
         l_linestatus
'''

In [23]:
resp, table = adapter.execute(query, fetch=True)

In [24]:
print(resp)

SUCCESS 4


In [35]:
table.print_table()

| L_RETURNFLAG | L_LINESTATUS |       SUM_QTY |       SUM_BASE_PRICE |       SUM_DISC_PRICE |           SUM_CHARGE | ... |
| ------------ | ------------ | ------------- | -------------------- | -------------------- | -------------------- | --- |
| A            | F            | 3,775,127,758 | 5,660,776,097,194.45 | 5,377,736,398,183... | 5,592,847,429,515... | ... |
| N            | F            |    98,553,062 |   147,771,098,385.98 | 140,384,965,965.035… | 145,999,793,032.776… | ... |
| N            | O            | 7,436,302,976 | 11,150,725,681,37... | 10,593,195,308,23... | 11,016,932,248,18... | ... |
| R            | F            | 3,775,724,970 | 5,661,603,032,745.34 | 5,378,513,563,915... | 5,593,662,252,666... | ... |
